# #30DayMapChallenge - Day 2: Lines

Create a quick and dirty line-plotting library that will be compatible with the 'renderer' we use later.

In [1]:
def round_point(pt):
    """Round both elements to nearest whole numbers."""
    return (round(pt[0]), round(pt[1]))


def naive_horizontal_line_between(start, end):
    """Naively draw a 'mostly' horizontal line between (not including)
    the start and end points."""

    # To make things easier, we're going to always draw from left to right.
    left = ()
    right = ()

    # Work out which of the start or end is the left or right.
    if start[0] < end[0]:
        left = start
        right = end
    else:
        left = end
        right = start
    
    # Do some basic "y = mx + c" calculations.
    dx = right[0] - left[0]
    dy = right[1] - left[1]
    m = dy / dx
    
    # Generate the list of pixels
    return [round_point((left[0] + x, left[1] + x * m)) for x in range(1, dx)]

def naive_vertical_line_between(start, end):
    """Naively draw a 'mostly' vertical line between (not including)
    the start and end points."""
    
    bottom = ()
    top = ()
    
    if start[1] < end[1]:
        bottom = start
        top = end
    else:
        bottom = end
        top = start
    
    dy = top[1] - bottom[1]
    dx = top[0] - bottom[0]
    
    m_prime = dx / dy
    
    return [round_point((bottom[0] + y * m_prime, bottom[1] + y)) for y in range(1, dy)]

def naive_line_between(start, end):
    """Draw a line between (but not including) the start and end points
    using a very naive (but simple) algorithm."""

    # Make sure we're only dealing with grid-aligned pixels.
    r_start = round_point(start)
    r_end = round_point(end)
    
    dx = abs(r_end[0] - r_start[0])
    dy = abs(r_end[1] - r_start[1])
    
    # We're only drawing the pixels between points, so they must be at least two pixels apart.
    if max(dy, dx) < 2:
        return []

    # If dy is larger, we're drawing a 'mostly' vertical line.
    if dy > dx:
        return naive_vertical_line_between(r_start, r_end)

    # If it's not, we're drawing a 'mostly' horizonal line.
    return naive_horizontal_line_between(r_start, r_end)    

Take some pre-computed routes and use our line-plotting library to generate all of the intermediate 'pixels'.

In [2]:
from pyproj import Transformer

# These are the pre-computed routes for two postcodes to their closest
# bus stop. We're going to render our output using Folium (LeafletJS),
# so our coordinates need to be in latLng format, not lngLat.
dirs = [
    [
        [57.492026, -4.244491],
        [57.492119, -4.244495],
        [57.4923,   -4.244322],
        [57.492923, -4.242583],
        [57.492947, -4.242439],
        [57.492932, -4.242223],
        [57.492897, -4.242134],
        [57.492481, -4.241533],
        [57.493061, -4.240132],
        [57.493213, -4.240332]
    ], [
        [57.49174,  -4.242951],
        [57.491854, -4.242911],
        [57.49195,  -4.242778],
        [57.492139, -4.242334],
        [57.492481, -4.241533],
        [57.493061, -4.240132],
        [57.493213, -4.240332]
    ]
]

# Transform WGS84 latLngs
gps2osgb = Transformer.from_crs('epsg:4326', 'epsg:27700')
osgb2gps = Transformer.from_crs('epsg:27700', 'epsg:4326')


heat = []

for dir_points in dirs:
    for i in range(0, len(dir_points) - 1):
        start = gps2osgb.transform(dir_points[i][0], dir_points[i][1])
        end = gps2osgb.transform(dir_points[i+1][0], dir_points[i+1][1])
        for point in naive_line_between(start, end):
            gps_point = osgb2gps.transform(point[0],point[1])
            heat.append([gps_point[0], gps_point[1], 0.1])
    for dir_point in dir_points:
        heat.append([dir_point[0], dir_point[1], 0.1])


Take our large collection of points and plot them on a map using a Folium HeatMap as the 'renderer'.

In [3]:
import folium
from folium.plugins import HeatMap

my_map = folium.Map(location=[57.492932, -4.242223], zoom_start=18, max_zoom=22)
HeatMap(heat, min_opacity=0.5, radius=25, blur=40).add_to(my_map)

my_map